# Deep Research with AutoGen

This notebook implements a multi-agent deep research system using AutoGen. It creates a team of specialized agents that collaborate to perform in-depth research on a given topic.

In [1]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.tools import TeamTool
# or import
from autogen_core.models import ModelInfo
import sys

sys.path.append(os.path.abspath(".."))
# Token usage tracking
from utilities.token_counting import (
    wrap_chat_client_for_tokens,
    print_token_usage_with_estimate,
    log_token_usage,
)

from utilities import (
    create_chat_completion_client,
    current_datetime_utc_tool,
    adjust_properties_for_fixed_response,
    get_tavily_search_tool,
    get_attributes,
    load_model_config,
)

#### Note: You need a tavily API key to use the web search tool!

In [2]:
model = load_model_config() 

# Check if web search is possible (i.e., if API key is set)
ALLOW_WEB_SEARCH = os.getenv("TAVILY_API_KEY", False)

Using model: gpt-4.1-nano type: openai


In [3]:
# Prepare tools
basic_tools = {current_datetime_utc_tool}
web_search_tools = set()
if ALLOW_WEB_SEARCH:
    # web_search_tool = [ await get_rag_web_browser_tool() ]
    web_search_tools = { await get_tavily_search_tool() }

Set the research topic
----------------------

In [4]:
# topic = "Impacts of Model Context Protocol on Agentic AI"
# topic = "What is Fraunhofer MEVIS?"
# topic = "What day and time is it in Berlin, Germany?"
# topic = "What is the wheather like in Berlin, Germany currently?"
topic = "What has Fraunhofer MEVIS done in the context of thoracic imaging?"

Select Model and Agent Configuration
-----------------------------

In [5]:

agentConfig = {
    "Researcher": dict(system_message="Gather and summarize factual info, add important web references.", tools=web_search_tools), 
    "FactChecker": dict(system_message="Use web search to verify facts and citations from the Researcher's response. For each found error, give a correct version and the web reference for verification.", tools=web_search_tools), 
    "Revisor": dict(system_message="Use the fact checker's results to revise the Researcher's report. Only if there were no fact issues, just repeat the Researcher's response. In both cases, add the fact checker's citations."),
    "Critic": dict(system_message="Critique clarity and logic."),
    "Summarizer": dict(system_message="Apply critic result to generate an improved response to the initial user question."),# Include the most important references."),
    "Editor": dict(system_message="Do some language polishing on the previous result to generate a final response to the initial question. Signal APPROVED when done."),
}

# For testing, we can fix an agent's response to a known answer
# adjust_properties_for_fixed_response(agentConfig["Researcher"], "Fraunhofer MEVIS is a research institute in Frankfurt for biological weapons. It was founded in 1995. See https://www.mefis.fraunhofer.de/en.html for more information.")

In [6]:
# Now create model client with token tracking
model_client = wrap_chat_client_for_tokens(create_chat_completion_client(model))

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    p = props.copy()
    p.setdefault("tools", set()).update(basic_tools)
    p.setdefault("reflect_on_tool_use", True)
    agents.append(
        AssistantAgent(
            name=agent_type,
            **p,
            model_client=model_client,
        )
    )

In [7]:
# Set up termination conditions
max_msgs = MaxMessageTermination(max_messages=20)
text_term = TextMentionTermination(text="APPROVED", sources=["Editor"])
termination = max_msgs | text_term

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)

In [8]:
# Create the host agent with deep dive capabilities and the host agent
deepdive_tool = TeamTool(team=team, name="DeepDive", description="Collaborative multi-agent deep dive")
host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[deepdive_tool],
    system_message="You have access to a DeepDive tool for in-depth research."
)

Start the Deep Dive
===================

In [9]:
async def run_deepdive(topic: str):
    start_time_s = timeit.default_timer()
    result = await team.run(task=f"Deep dive on: {topic}")
    # Try native usage first; fallback to estimation
    print_token_usage_with_estimate(model_client, getattr(result, 'messages', None), agentConfig.keys(), model_info=model)
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 DeepDive result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await run_deepdive(topic)

Estimated token usage -> prompt: 2158, completion: 2665, total: 4823 (method=heuristic)
Approximate cost for this run: $0.0006 USD (input: $0.0001, output: $0.0005)
🔍 DeepDive result (took 20.91 sec.) resulted in 11 messages.


Explore Results
===============

In [10]:
# Display token usage after the deep dive
print("=== Token Usage Summary ===")
print_token_usage_with_estimate(model_client, getattr(result, 'messages', None), model_info=model)

=== Token Usage Summary ===
Estimated token usage -> prompt: 2158, completion: 2665, total: 4823 (method=heuristic)
Approximate cost for this run: $0.0006 USD (input: $0.0001, output: $0.0005)


In [11]:
# Display the full result structure
print("Result type:", type(result))

# print("\nResult attributes:")
# print(get_attributes(result))
# print("\nMessage attributes:")
# print(get_attributes(result.messages[-1]))

print("Message Sequence:")
for i, m in enumerate(result.messages):  
    print(f"\n----------------------------------------------------------------")
    print(f"MESSAGE {i} [{m.source}]:\n{m.content}") # type: ignore

Result type: <class 'autogen_agentchat.base._task.TaskResult'>
Message Sequence:

----------------------------------------------------------------
MESSAGE 0 [user]:
Deep dive on: What has Fraunhofer MEVIS done in the context of thoracic imaging?

----------------------------------------------------------------
MESSAGE 1 [Researcher]:
[FunctionCall(id='call_EC7lyX9BqhZLV7fH2g8S0EDx', arguments='{"query": "Fraunhofer MEVIS thoracic imaging"}', name='tavily_search'), FunctionCall(id='call_VfnokBpeC4EPgewEWUFGdnrS', arguments='{"query": "Fraunhofer MEVIS contributions to thoracic imaging"}', name='tavily_search')]

----------------------------------------------------------------
MESSAGE 2 [Researcher]:
[FunctionExecutionResult(content='Search Query: Fraunhofer MEVIS thoracic imaging\nResults:\n1. Solutions for Cardiac Imaging - Fraunhofer MEVIS\nURL: https://www.mevis.fraunhofer.de/en/solutionpages/solutions-for-cardiac-imaging.html\nSnippet: Fraunhofer MEVIS develops software solutions fo

In [12]:
# Clean up resources
await model_client.close()

In [13]:
import re
from autogen_agentchat.messages import BaseTextChatMessage

print("Stop reason:", result.stop_reason)
assert isinstance(result.messages[-1], BaseTextChatMessage)
last_message = result.messages[-1].content

print("Message Sequence:")
agent_names = list(agentConfig.keys())
pattern = re.compile(r'(' + '|'.join(re.escape(n) for n in agent_names) + r'):')
for match in pattern.finditer(last_message):
    print(" -", match.group(1))

print("\n", last_message)  # Print the content of the last message

Stop reason: Maximum number of turns 6 reached.
Message Sequence:

 Fraunhofer MEVIS has made notable advances in thoracic imaging by developing sophisticated software for lung and cardiac analysis. Their innovations include automated tools for CT and MRI lung disease detection and monitoring, rapid and precise non-rigid lung registration techniques enabling effective comparison of thoracic scans over time, and multimodal cardiac imaging solutions for structure quantification and early diagnosis. They actively participate in scientific communities like MICCAI, sharing advancements such as interactive 3D visualizations of the heart, and their solutions are integrated into clinical workflows through collaborations with partners like Veolity. These efforts establish Fraunhofer MEVIS as a leader in advancing thoracic diagnostic and research capabilities.


In [14]:
# Optional: append token usage (estimated or native) to a CSV log for tracking across runs
# log_token_usage imported from token_counting
log_token_usage(topic, model_client, getattr(result, 'messages', None), agentConfig.keys(), model_info=model)

Logged token usage to token_usage_log.csv: {'timestamp': '2025-09-22T09:55:51.538019+00:00', 'topic': 'What has Fraunhofer MEVIS done in the context of thoracic imaging?', 'prompt': 2158, 'completion': 2665, 'total': 4823, 'method': 'heuristic', 'cost_usd': 0.000641, 'input_cost_usd': 0.000108, 'output_cost_usd': 0.000533}


{'timestamp': '2025-09-22T09:55:51.538019+00:00',
 'topic': 'What has Fraunhofer MEVIS done in the context of thoracic imaging?',
 'prompt': 2158,
 'completion': 2665,
 'total': 4823,
 'method': 'heuristic',
 'cost_usd': 0.000641,
 'input_cost_usd': 0.000108,
 'output_cost_usd': 0.000533}